In [1]:
%load_ext autoreload
%autoreload 2

import os,sys
parent_dir = os.path.abspath('..')
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)
import util as yu
from util import *
import util_Nsgm as yu2

yu.setpath('analysis_noNsgm')

In [2]:
path='/capstor/store/cscs/userlab/lp139/lyan/code/projectData/01_Nsgm/data/Nsgm_4src_16,18,20_cB211.072.64.h5'
# yu.print_hdf5_structure(path)

opabs=['g;0,0,0;G1g;a;l1;N_g;0,0,0;G1g;a;l1;N', 'g;0,0,0;G1g;a;l2;N_g;0,0,0;G1g;a;l2;N']
with h5py.File(path) as f:
    dic=yu.list2dic(yu.decodeList(f['diags/N/opabs'][:]))
    t=f['diags/N/data/p_p'][:]
    t=np.mean([t[:,:,dic[opab]] for opab in opabs],axis=0)
    t=np.real(t)
    c2pt=yu.jackknife(t)

In [9]:
ens='b'
mul=yu.ens2amul[ens]

tfs=[8,10,12,14,16,18,20]

tf2c3pt_conn={}
tf2c3pt_disc={}
opabs=['g;0,0,0;G1g;a;l1;N_g;0,0,0;G1g;a;l1;N', 'g;0,0,0;G1g;a;l2;N_g;0,0,0;G1g;a;l2;N']
path='/capstor/store/cscs/userlab/lp139/lyan/code/projectData/01_Nsgm/data/Nsgm_4src_16,18,20_cB211.072.64.h5'
with h5py.File(path) as f:
    cfgs=yu.decodeList(f['cfgs'])
    for tf in tfs:
        iins=yu.decodeList(f['diags/NJN/inserts']).index('id')
        dic=yu.list2dic(yu.decodeList(f['diags/NJN/opabs'][:]))
        t=f[f'diags/NJN/data/p_j+_p_deltat_{tf}'][:,:,:,iins]
        t=np.mean([t[:,:,dic[opab]] for opab in opabs],axis=0)
        t=np.real(t*mul)
        t=yu.jackknife(t)
        tf2c3pt_conn[tf]=t

        iins=yu.decodeList(f['diags/N-j/inserts']).index('id')
        dic=yu.list2dic(yu.decodeList(f['diags/N-j/opabs'][:]))
        t=f[f'diags/N-j/data/p_j+_p_deltat_{tf}'][:,:,:,iins]
        t=np.mean([t[:,:,dic[opab]] for opab in opabs],axis=0)
        t=np.real(t*mul)
        t=yu.jackknife(t)
        
        tvev=np.real(f['VEV/j/data/j+'][:,iins]*mul)
        tvev=yu.jackknife(tvev)
        
        tf2c3pt_disc[tf]=t-c2pt[:,tf:tf+1]*tvev[:,None]

cfg2old=lambda cfg: cfg[1:]+'_r'+{'a':'0','b':'1','c':'2','d':'3'}[cfg[0]]
cfg2new=lambda cfg: {'0':'a','1':'b','2':'c','3':'d'}[cfg[-1]] + cfg[:4]
path='/capstor/store/cscs/userlab/lp139/lyan/code/projectData_old/01_Nsgm/cB211.072.64_base/data_from_others/thrp-conn.h5'
with h5py.File(path) as f:
    cfgs_old=[cfg2old(cfg) for cfg in cfgs]

    tf2c3pt_conn_more={tf:np.real(yu.jackknife([f[f'gS/up/dt{tf}/{cfg}'][:] + f[f'gS/dn/dt{tf}/{cfg}'][:] for cfg in cfgs_old])*mul) for tf in tfs}
    tf2c2pt={tf:np.real(yu.jackknife([f[f'dt{tf}_twop/{cfg}'][()] for cfg in cfgs_old])) for tf in tfs}
    
tf2c3pt={}
for tf in tfs:
    tf2c3pt[tf]=tf2c3pt_conn_more[tf]+tf2c3pt_disc[tf]
    # if True:
    #     tf2c3pt[tf]=(tf2c3pt[tf]+tf2c3pt[tf][:,::-1])/2

In [18]:
dic={
    'c2pt':c2pt,
    'tf2c3pt':tf2c3pt,
    'mul':mul
}
yu.save_pkl('/capstor/store/cscs/userlab/lp139/lyan/code/temp/data_sgm_B64.pkl',dic)

In [20]:
c2pt.shape

(732, 32)

In [19]:
dic.keys()

dict_keys(['c2pt', 'tf2c3pt', 'mul'])